# Heart Failure PAF Calculations 

This workbook is designed to find the PAF for BMI to heart failure ONLY while NOT considering risk correlation to mediation. The more complex versiont that includes these considerations is in another workbook. 

In [11]:
from vivarium import InteractiveContext, Artifact

from datetime import datetime, timedelta
from pathlib import Path
import itertools
import matplotlib.pyplot as plt
import ipywidgets
import pandas as pd, numpy as np
pd.set_option('display.max_rows', 60)
import seaborn as sns

import numpy as np
import researchpy as rp
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib.backends.backend_pdf import PdfPages
import scipy.stats
from typing import List, Tuple, Union
import scipy.stats as sp

In [2]:
path = Path('/ihme/homes/lutzes/vivarium_nih_us_cvd/src/vivarium_nih_us_cvd/model_specifications/nih_us_cvd.yaml')

In [3]:
sim = InteractiveContext(Path(path), setup=False)
sim.configuration.update({
                          'population':
                              {'population_size': 10_000,
                              },
                          'time':
                              {'start':
                                  {'year': 2021
                                  }
                              }
                          }
                        )
sim.setup()

2023-01-17 13:47:03.084 | DEBUG    | vivarium.framework.values:register_value_modifier:392 - Registering metrics.1.population_manager.metrics as modifier to metrics
2023-01-17 13:47:03.179 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:66 - Running simulation from artifact located at /ihme/costeffectiveness/artifacts/vivarium_nih_us_cvd/alabama.hdf.
2023-01-17 13:47:03.182 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:67 - Artifact base filter terms are ['draw == 0'].
2023-01-17 13:47:03.185 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:68 - Artifact additional filter terms are None.
2023-01-17 13:47:03.541 | DEBUG    | vivarium.framework.values:_register_value_producer:338 - Registering value pipeline cause_specific_mortality_rate
2023-01-17 13:47:03.542 | DEBUG    | vivarium.framework.values:_register_value_producer:338 - Registering value pipeline mortality_rate
2023-01-17 13:47:03.896 | DEBUG    | vivarium.framework.values:_regis

In [4]:
pop0 = sim.get_population()
pop0.head()

,tracked,location,sex,alive,entrance_time,age,exit_time,years_of_life_lost,cause_of_death,years_lived_with_disability,...,outreach,sbp_medication,sbp_multiplier,ldlc_medication_adherence,polypill,sbp_medication_adherence,ldlc_medication,ldlc_multiplier,visit_type,scheduled_date
0,True,Alabama,Female,alive,2020-12-04,47.388882,NaT,0.0,not_dead,0.0,...,cat2,no_treatment,1.0,cat3,cat2,cat3,no_treatment,1.000,none,NaT
1,True,Alabama,Male,alive,2020-12-04,41.564194,NaT,0.0,not_dead,0.0,...,cat2,no_treatment,1.0,cat3,cat2,cat3,no_treatment,1.000,none,NaT
2,True,Alabama,Female,alive,2020-12-04,53.155826,NaT,0.0,not_dead,0.0,...,cat2,one_drug_half_dose_efficacy,1.0,cat3,cat2,cat1,medium_intensity,1.362,none,NaT
3,True,Alabama,Male,alive,2020-12-04,30.457692,NaT,0.0,not_dead,0.0,...,cat2,no_treatment,1.0,cat1,cat2,cat3,no_treatment,1.000,none,NaT
4,True,Alabama,Female,alive,2020-12-04,19.554806,NaT,0.0,not_dead,0.0,...,cat2,no_treatment,1.0,cat1,cat2,cat3,no_treatment,1.000,none,NaT


In [5]:
pop1 = pop0[['sex','alive','age']]

In [6]:
#pop0.columns

In [7]:
#sim.list_values()

In [6]:
data1 = pd.concat([pop1,
                   sim.get_value('high_body_mass_index_in_adults.propensity')(pop0.index).rename('bmi_propensity'),
                   sim.get_value('high_body_mass_index_in_adults.exposure')(pop0.index).rename('bmi_exposure'),
                  ], axis=1)
data1.head()

,sex,alive,age,bmi_propensity,bmi_exposure
0,Female,alive,47.388882,0.208828,23.564850
1,Male,alive,41.564194,0.358604,25.661107
2,Female,alive,53.155826,0.531571,34.110372
3,Male,alive,30.457692,0.388049,26.473747
4,Female,alive,19.554806,0.672873,5.000000


In [7]:
data1 = data1.loc[(data1.age > 25) & (data1.alive == 'alive')]

## Adding RR and TMREL so I can find the individual RR for a Simulant

Note that while this method will randomly generate data, I eventually need to match the RR's to the simulant for the specific draw. As HF has not been added yet, I am not sure I can do this currently.

Also, we might just have the engineers add this to their initialization tasks instead! 

In [12]:
def get_lognorm_from_quantiles(median: float, lower: float, upper: float,
                               quantiles: Tuple[float, float] = (0.025, 0.975)) -> sp.lognorm:
    mu = np.log(median)
    stdnorm_quantiles = sp.norm.ppf(quantiles)
    norm_quantiles = np.log([lower, upper])
    sigma = (norm_quantiles[1] - norm_quantiles[0]) / (stdnorm_quantiles[1] - stdnorm_quantiles[0])
    return sp.lognorm(s=sigma, scale=median)

In [13]:
test2 = get_lognorm_from_quantiles(1.14,1.12,1.16).rvs(size=len(data1))

In [14]:
data1['RR'] = test2
data1.head()

,sex,alive,age,bmi_propensity,bmi_exposure,RR
0,Female,alive,47.388882,0.208828,23.564850,1.128200
1,Male,alive,41.564194,0.358604,25.661107,1.158464
2,Female,alive,53.155826,0.531571,34.110372,1.147754
3,Male,alive,30.457692,0.388049,26.473747,1.123268
6,Male,alive,33.645113,0.199137,22.520013,1.148181


In [16]:
np.random.seed(123)
test3 = np.random.uniform(20,25,len(data1))
test3

array([23.48234593, 21.43069667, 21.13425727, ..., 23.09954019,
       24.65176124, 22.11263259])

In [17]:
data1['TMREL'] = test3
data1.head()

,sex,alive,age,bmi_propensity,bmi_exposure,RR,TMREL
0,Female,alive,47.388882,0.208828,23.564850,1.128200,23.482346
1,Male,alive,41.564194,0.358604,25.661107,1.158464,21.430697
2,Female,alive,53.155826,0.531571,34.110372,1.147754,21.134257
3,Male,alive,30.457692,0.388049,26.473747,1.123268,22.756574
6,Male,alive,33.645113,0.199137,22.520013,1.148181,23.597345


## Finding the individual simulant's RR and PAF

In [23]:
max((data1['bmi_exposure'].iloc[4] - data1['TMREL'].iloc[4]),0)

max((data1['bmi_exposure'] - data1['TMREL']),0)

0

In [28]:
for i in range(len(data1)):
    data1['sim_RR'].iloc[i] = data1['RR'].iloc[i]**[max((data1['bmi_exposure'].iloc[i] - data1['TMREL'].iloc[i]),0)]
data1

/tmp/ipykernel_2756657/1010717381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['sim_RR'].iloc[i] = data1['RR'].iloc[i]**[max((data1['bmi_exposure'].iloc[i] - data1['TMREL'].iloc[i]),0)]


,sex,alive,age,bmi_propensity,bmi_exposure,RR,TMREL,sim_RR
0,Female,alive,47.388882,0.208828,23.564850,1.128200,23.482346,1.010002
1,Male,alive,41.564194,0.358604,25.661107,1.158464,21.430697,1.863158
2,Female,alive,53.155826,0.531571,34.110372,1.147754,21.134257,5.978698
3,Male,alive,30.457692,0.388049,26.473747,1.123268,22.756574,1.540477
6,Male,alive,33.645113,0.199137,22.520013,1.148181,23.597345,1.000000
...,...,...,...,...,...,...,...,...
9994,Male,alive,82.973183,0.491704,26.062478,1.129068,22.670537,1.509466
9995,Female,alive,31.875800,0.356089,27.354193,1.138290,20.679220,2.374038
9996,Male,alive,69.906989,0.013521,19.319172,1.139587,23.099540,1.000000
9998,Female,alive,77.277694,0.623941,31.170490,1.149427,24.651761,2.478911


In [30]:
data1['PAF'] = (data1['sim_RR'] - 1) / data1['sim_RR']
data1

,sex,alive,age,bmi_propensity,bmi_exposure,RR,TMREL,sim_RR,PAF
0,Female,alive,47.388882,0.208828,23.564850,1.128200,23.482346,1.010002,0.009903
1,Male,alive,41.564194,0.358604,25.661107,1.158464,21.430697,1.863158,0.463277
2,Female,alive,53.155826,0.531571,34.110372,1.147754,21.134257,5.978698,0.832739
3,Male,alive,30.457692,0.388049,26.473747,1.123268,22.756574,1.540477,0.350850
6,Male,alive,33.645113,0.199137,22.520013,1.148181,23.597345,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...
9994,Male,alive,82.973183,0.491704,26.062478,1.129068,22.670537,1.509466,0.337514
9995,Female,alive,31.875800,0.356089,27.354193,1.138290,20.679220,2.374038,0.578777
9996,Male,alive,69.906989,0.013521,19.319172,1.139587,23.099540,1.000000,0.000000
9998,Female,alive,77.277694,0.623941,31.170490,1.149427,24.651761,2.478911,0.596597
